In [1]:
import torch
import os
import json

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.0.0+cu117
True


In [3]:
# load some questions
print(os.listdir("../data"))

with open('../data/question_db_3.json', 'r') as f:
    questions = json.load(f)
len(questions)

['logs', 'question_db_2.json', 'question_db.json', 'question_db_3.json', 'qa']


13108

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from transformers.models.t5.modeling_t5 import T5ForConditionalGeneration

# t5_qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-xxl-ssm-tqa")
t5_qa_model = T5ForConditionalGeneration.from_pretrained("google/t5-xxl-ssm-tqa")
t5_tok = AutoTokenizer.from_pretrained("google/t5-xxl-ssm-tqa")


In [5]:
t5_qa_model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=4096, out_features=4096, bias=False)
              (k): Linear(in_features=4096, out_features=4096, bias=False)
              (v): Linear(in_features=4096, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 64)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=4096, out_features=10240, bias=False)
              (wi_1): Linear(in_features=4096, out_features=10240, bias=False)
     

In [6]:
print(type(t5_qa_model))

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


In [7]:
def get_answer(question_text):
    input_ids = t5_tok(question_text, return_tensors="pt").input_ids.cuda()
    with torch.no_grad():
        gen_output = t5_qa_model.generate(input_ids, do_sample=True)[0]
    return t5_tok.decode(gen_output, skip_special_tokens=True)

# input_ids = t5_tok("When was Franklin D. Roosevelt born?", return_tensors="pt").input_ids
# gen_output = t5_qa_model.generate(input_ids)[0]
# print(t5_tok.decode(gen_output, skip_special_tokens=True))
# get_answer('When was Fralin D. Roosevelt born?')
# get_answer("When was Franklin D. Roosevelt born?")

In [8]:
for q in questions[:5]:
    print(q['question'])
    print(get_answer(q['question']))
    print()


Name A Number That’s Associated With Being Lucky Or Unlucky
13

Name The Most Used Piece Of Furniture In A House.
Chair

Name A Job In Which An Attractive Person Probably Makes Better Tips.
Cab Driver

Name A Prop used By Tap Dancers
Tap Shoes

If You Drew Homer Simpson’s Name In A Secret Santa Exchange, What Would You Buy Him?
A Bargain Basement



In [9]:
question_text =  questions[2]['question']
print(question_text)
for _ in range(50):
    print(get_answer(question_text))



Name A Job In Which An Attractive Person Probably Makes Better Tips.
Waitress
Waitress
Cab Driver
Cab Driver
Cabdriver
Waitress
Cab Driver
Waitress
Cab Driver
Waitress
Waitress
Cab Driver
Cab Driver
Cab Driver
Cab Driver
Cab Driver
Cab Driver
Cab Driver
Waitress
Server
Cab Driver
Waitress
Cab Driver, Waitress, Bartender, Nurse, Hairdresser, Makeup Artist
Cab Driver
Cab Driver
Cab Driver
Waitress
Cab Driver
Waitress
Cab Driver
Cab Driver
Cab Driver
Waitress
Cab Driver
Cab Driver
Cabdriver
Cab Driver
Waitress
Waitress
Tipsy Cab Driver
Cab Driver
Waitress
Cab Driver
Cab Driver
Waitress
Waitress
Waitress
Cab Driver
Bartender
Cab Driver
